In [30]:
from typing import cast

import networkx as nx
import numpy as np
import psycopg2
import torch
import polars as pl
from dotenv import dotenv_values
from locus.data.QuadTree import CellState
from locus.models.dataloader import LDoGIDataLoader
from locus.models.dataset import LDoGIDataset
from locus.utils.paths import PROCESSED_DATA_DIR, PROJECT_ROOT, SQL_DIR
from networkx import DiGraph
from torch.utils.data import BatchSampler, DataLoader, RandomSampler
from geopy import distance


In [31]:
config = dotenv_values(PROJECT_ROOT / ".env")

conn = psycopg2.connect(
    host=config["DB_HOST"],
    port=config["DB_PORT"],
    dbname=config["DB_NAME"],
    user=config["DB_USER"],
    password=config["DB_PASSWORD"],
)
cur = conn.cursor()

In [32]:
with open(SQL_DIR / "select_max_id.sql") as f:
    cur.execute(f.read())

# Retrieve query results
max_id = cur.fetchall()[0][0]
max_id

4233900

In [33]:
sql_string = 'select dataset.latitude, dataset.longitude from dataset where id > {} and id < {};'

In [34]:
sum_dist_sqr = 0
sum_dist = 0

In [35]:
average_lat = 35.428497693121706
average_lon = -16.918947023563472

In [36]:
batch_size = 10000
min_pct = 0.7
max_pct = 0.9

In [37]:
for i in range(int(max_id*min_pct), int(max_id*max_pct), batch_size):
    data = pl.read_database(sql_string.format(i, i+batch_size), conn)
    for d in data.rows(named=True):
        dist = distance.great_circle((average_lat, average_lon), (d['latitude'], d['longitude'])).km
        sum_dist_sqr += dist**2
        sum_dist += dist


In [38]:
mse = sum_dist_sqr / int(max_id*(max_pct - min_pct))
me = sum_dist / int(max_id*(max_pct - min_pct))

In [39]:
mse

49788457.26168653

In [40]:
me

5866.976065916786